## Computational graph

In [11]:
import tensorflow as tf

def linear_layer(x):
    return 3 * x + 2

@tf.function
def simple_nn(x):
    return tf.nn.relu(linear_layer(x))

def simple_function(x):
    return 3 * x

`tf.function`을 사용시에, 하나의 주 함수에만 어노테이션을 달면 거기에 호출된 다른 모든 함수는 자동으로 투명하게 최적화된 계산 그래프로 변환된다. 

In [12]:
simple_nn # 텐서플로 내부와 상호작용하는 특수 handler

In [13]:
simple_function # 일반 파이썬 handler

<function __main__.simple_function(x)>

In [17]:
# @tf.function 데코레이터 사용과 미사용 속도차이
import timeit

cell = tf.keras.layers.LSTMCell(100)

@tf.function
def fn(input, state):
    return cell(input, state)

input = tf.zeros([100, 100])
state = [tf.zeros([100, 100])] * 2

cell(input, state)
fn(input, state)

graph_time = timeit.timeit(lambda : cell(input, state), number = 100)
auto_graph_time = timeit.timeit(lambda : fn(input, state), number = 100)
print("graph_time : ", graph_time)
print("auto_graph_time : ", auto_graph_time)

graph_time :  0.09546189999991839
auto_graph_time :  0.03926569999998719


## Keras model api

In [18]:
# 함수형 api
def build_model():
    # 가변길이 정수 시퀀스
    text_input_a = tf.keras.Input(shape = (None, ), dtype = "int32")
    # 가변길이 정수 시퀀스
    text_input_b = tf.keras.Input(shape = (None, ), dtype = "int32")
    # 1000개의 고유 단어를 128차원 벡터에 매핑하여 임베딩
    shared_embedding = tf.keras.layers.Embedding(1000, 128)
    
    # 두 입력을 인코딩하고자 동일 임베딩 계층 재사용
    encoded_input_a = shared_embedding(text_input_a)
    encoded_input_b = shared_embedding(text_input_b)
    
    # 2개의 로지스틱 예측
    prediction_a = tf.keras.layers.Dense(1, activation = "sigmoid",
                                        name = 'prediction_a')(encoded_input_a)
    prediction_b = tf.keras.layers.Dense(1, activation = "sigmoid",
                                        name = 'prediction_b')(encoded_input_b)
    
    model = tf.keras.Model(inputs = [text_input_a, text_input_b],
                           outputs = [prediction_a, prediction_b])
    tf.keras.utils.plot_model(model, to_file = "shared_model.png")
    
build_model()

In [20]:
from tensorflow.keras import layers

# 서브클래싱 api
class MyLayer(layers.Layer):
    def __init(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.kernel = self.add_weight(name = "kernel",
                                      shape = (input_shape[1], self.output_dim),
                                      initializer = "uniform",
                                      trainable = True)
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

## useful Callbacks
* tf.keras.callbacks.ModelCheckpoint
* tf.keras.callbacks.LearningRateScheduler
* tf.keras.callbacks.EarlyStopping
* tf.keras.callbacks.TensorBoard

## 모델, 가중치 저장
* model.save_weights("")
* model.load_weights("")
* model.save("name.h5")
* model = tf.keras.models.load_model("name.h5")
* json_string = model.to_json()
* model = tf.keras.models.model_from_json(json_string)

## tensorflow datasets
* 원칙적인 방식으로 입력 데이터를 처리하는데 필요한 라이브러리
---
1. 생성 :
  1. `from_tensor_slices()` : 개별(또는 다중) 넘파이배열(또는 텐서)를 받고 배치를 지원
  2. `from tensors()` : 배치를 지원하지 않음
  3. `from_generator()` : 제너레이터 함수에서 입력을 취함
2. 변환 :
  1. `batch()` : 순차적으로 데이터셋을 지정된 사이즈의 배치로 분할
  2. `repeat()` : 데이터를 복제
  3. `shuffle()` : 데이터 셔플
  4. `map()` : 데이터에 함수 적용
  5. `filter()` : 데이터를 거르고자 함수를 적용
3. 반복자 :
  1. `next_batch = iterator.get_next()`

In [21]:
import tensorflow_datasets as tfds

builders = tfds.list_builders()
print(builders)

['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'amazon_us_reviews', 'anli', 'arc', 'bair_robot_pushing_small', 'bccd', 'beans', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'blimp', 'bool_q', 'c4', 'caltech101', 'caltech_birds2010', 'caltech_birds2011', 'cars196', 'cassava', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cfq', 'cherry_blossoms', 'chexpert', 'cifar10', 'cifar100', 'cifar10_1', 'cifar10_corrupted', 'citrus_leaves', 'cityscapes', 'civil_comments', 'clevr', 'clic', 'clinc_oos', 'cmaterdb', 'cnn_dailymail', 'coco', 'coco_captions', 'coil100', 'colorectal_histology', 'colorectal_histology_large', 'common_voice', 'coqa', 'cos_e', 'cosmos_qa', 'covid19sum', 'crema_d', 'curated_breast_imaging_ddsm', 'cycle_gan', 'dart', 'davis', 'deep_weeds', 'definite_pronoun_resolution', 'dementiabank', 'diabetic_retinopathy_detection', 'div2k', 'dmlab', 'downsampled_imagenet', 'drop', 'dsprites', 'dtd', 

In [22]:
data, info = tfds.load("rock_paper_scissors", with_info = True)
train_data, test_data = data['train'], data['test']

print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling rock_paper_scissors-train.tfrecord...:   0%|          | 0/2520 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling rock_paper_scissors-test.tfrecord...:   0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to C:\Users\user\tensorflow_datasets\rock_paper_scissors\3.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_path='C:\\Users\\user\\tensorflow_datasets\\rock_paper_scissors\\3.0.0',
    download_size=219.53 MiB,
    dataset_size=219.23 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Data

In [25]:
import numpy as np

num_items = 100
num_list = np.arange(num_items)

num_list_dataset = tf.data.Dataset.from_tensor_slices(num_list)

In [30]:
datasets, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)
train_dataset = datasets['train']
train_dataset = train_dataset.batch(5).shuffle(50).take(2)

# for data in train_dataset:
#     print(data)

## custom gradient calcul

In [ ]:
@tf.function

def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        regularization_loss = //
        pred_loss = //
        total_loss = pred_loss + regularization_loss
    
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
for epoch in range(NUM_EPOCHS):
    for inputs, labels in train_data:
        train_step(inputs, labels)
    print("Finished epoch", epoch)

# 효율적인 텐서플로 2.x 사용
---
1. 웬만하면 고수준 api 사용하기
2. tf.function 데코레이터 추가하여 오토그래프로 그래프모드에서 효율적으로 수행되도록
3. 파이썬 객체를 사용해 변수와 손실을 추적 -> tf.Variable 사용
4. 데이터 입력으로는 tf.data 데이터셋을 사용하고 이 객체를 바로 tf.keras.Model.fit에 제공
5. 가능하면 tf.layers 모듈을 사용해 순차형, 함수형 api로 사전정의된 블록 조합
6. GPU, CPU, 여러 플랫폼으로의 분산전력 사용을 고려